# Webscraping de supermecados. Caso tortilla

In [16]:
#Importamos librerías
import pandas as pd
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random
import json
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
#Importar webdriverwait
from selenium.webdriver.support.ui import WebDriverWait
#Importar expected_conditions
from selenium.webdriver.support import expected_conditions as EC

## Chedraui

In [17]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-geolocation')
driver=webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
url="https://www.chedraui.com.mx/tortillas?_q=tortillas&map=ft"
driver.get(url)

#contenido dinamico	
# wait for the element containing the dynamic content to appear
dynamic_content_element = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, '//span[@class="vtex-product-summary-2-x-currencyContainer"]'))
)

# get the current page height
last_height = driver.execute_script("return document.body.scrollHeight")

# scroll to the bottom of the page to load any additional dynamic content
scroll_speed = 1000
while True:
    driver.execute_script("window.scrollBy(0, " + str(scroll_speed) + ");")
    driver.execute_script("window.scrollBy(0, " + str(scroll_speed) + ");")
    driver.execute_script("window.scrollBy(0, " + str(scroll_speed) + ");")
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height





In [18]:
#Hacer webscraping
soup=BeautifulSoup(driver.page_source,'html.parser')
productos=soup.find_all('span',class_='vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body')
#Dormir 5 segundos
pesos=soup.find_all('span',class_='vtex-product-summary-2-x-currencyInteger')
centavos=soup.find_all('span',class_='vtex-product-summary-2-x-currencyFraction')
#data=soup.find_all('div',class_="render-container render-route-store-search")

In [19]:
#Crear dataframe

chedraui=pd.DataFrame({'producto':productos,'pesos':pesos,'centavos':centavos})
#Sacar los datos de los tags
chedraui['producto']=chedraui['producto'].apply(lambda x: x.text)
chedraui['pesos']=chedraui['pesos'].apply(lambda x: x.text)
chedraui['centavos']=chedraui['centavos'].apply(lambda x: x.text)
#Crear columna precio
chedraui['precio']=chedraui['pesos']+'.'+chedraui['centavos']
#Eliminar columnas pesos y centavos
chedraui.drop(['pesos','centavos'],axis=1,inplace=True)
#Transformar a float
chedraui['precio']=chedraui['precio'].astype(float)
#Incorporar el nombre de la tienda
chedraui['tienda']='Chedraui'
#Agregar hora de consulta
chedraui['hora_consulta']=datetime.datetime.now()
chedraui

,producto,precio,tienda,hora_consulta
0,Tortilla de Maíz Amarillo Kg,12.40,Chedraui,2023-03-16 15:12:01.043505
1,Tortillas Blanca Kg,12.40,Chedraui,2023-03-16 15:12:01.043505
2,Tortilla Harina Tía Rosa 50% Menos Grasa Paque...,18.45,Chedraui,2023-03-16 15:12:01.043505
3,Tortilla Del Barrio Harina Trigo 315g,18.05,Chedraui,2023-03-16 15:12:01.043505
4,Tortillas de Harina Integral Tia Rosa 255g,26.00,Chedraui,2023-03-16 15:12:01.043505
5,Tortilla Harina Tia Rosa Burrito 10 Piezas500g,39.00,Chedraui,2023-03-16 15:12:01.043505
6,Tortilla Selecto Harina Trigo 250g,17.50,Chedraui,2023-03-16 15:12:01.043505
7,Tortilla de Maiz Guerreras 800g,29.70,Chedraui,2023-03-16 15:12:01.043505
8,Tortilla De Maíz Mission Blanca 550g,25.95,Chedraui,2023-03-16 15:12:01.043505
9,Tortillas Nopalia Nopal 450g,31.40,Chedraui,2023-03-16 15:12:01.043505


## Soriana

In [20]:
#Soriana
import json
url_soriana="https://www.soriana.com/buscar?q=tortilla&cid=&search-button=&page="
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

data_cbt=[]


for i in range(1,4):
    
    url=url_soriana+str(i)
    response=requests.get(url,headers=headers)
    soup=BeautifulSoup(response.text,'html.parser')
    data=soup.find_all('a',class_='link plp-link font-primary--medium product-tile--link ellipsis-product-name')
    #Usar get para obtener el atributo href
    data_cbt.extend([x.get('data-cbt') for x in data])

products = []

for string in data_cbt:
    data = json.loads(string)
    product = data['ecommerce']['click']['products'][0]
    products.append(product)
    

In [21]:
soriana=pd.DataFrame(products)
#Dejar name y price y cambiar el nombre de price a precio y name a producto
soriana=soriana[['name','price']]
soriana.columns=['producto','precio']
soriana['tienda']='Soriana'
#Agregar hora de la consulta
soriana['hora_consulta']=datetime.datetime.now()
soriana

,producto,precio,tienda,hora_consulta
0,Tortilla de Maíz Azul 1 Kg,22.0,Soriana,2023-03-16 15:12:13.286630
1,Tortilla de Maíz Nopal y Linaza 1 Kg,22.0,Soriana,2023-03-16 15:12:13.286630
2,Sopes de Maíz Amarillo 8 Piezas,23.0,Soriana,2023-03-16 15:12:13.286630
3,Tortilla Amarilla Taquera 1 Kg,22.0,Soriana,2023-03-16 15:12:13.286630
4,Tortilla para Flauta 1 Kg,22.0,Soriana,2023-03-16 15:12:13.286630
...,...,...,...,...
83,Tortillas Fritas Fresh Gourmet Estilo Santa Fé...,38.9,Soriana,2023-03-16 15:12:13.286630
84,Tortillas de Trigo Entero Food For Life 340 Gr,167.4,Soriana,2023-03-16 15:12:13.286630
85,Tortilla de Nopal Muy Natural con Chía y Ajonj...,38.0,Soriana,2023-03-16 15:12:13.286630
86,Tortilla de Nopal Muy Natural con Chía y Aránd...,38.0,Soriana,2023-03-16 15:12:13.286630


## Walmart

In [22]:
# Walmart
url_walmart="https://super.walmart.com.mx/search?q=tortillas&page="
totales=[]

for i in range(1,8):
    url=url_walmart+str(i)
    print(url)
    response=requests.get(url,headers=headers)
    soup=BeautifulSoup(response.text,'html.parser')
    #Aplicaar prettify para ver el html
    html=soup.prettify()
    #Hacer el html mas legible
    html=BeautifulSoup(html,'html.parser')
    data=html.find_all('span',class_='w_A6')
    totales.extend(data)



https://super.walmart.com.mx/search?q=tortillas&page=1
https://super.walmart.com.mx/search?q=tortillas&page=2
https://super.walmart.com.mx/search?q=tortillas&page=3
https://super.walmart.com.mx/search?q=tortillas&page=4
https://super.walmart.com.mx/search?q=tortillas&page=5
https://super.walmart.com.mx/search?q=tortillas&page=6
https://super.walmart.com.mx/search?q=tortillas&page=7


In [23]:
 #Borrar espacios en blanco
limpia=[x.text.strip() for x in totales]
#Eliminar aquello que diga "costaba"
limpia=[x.replace('costaba','') for x in limpia]
#Eliminar vacios
limpia=[x for x in limpia if x!='']
#Extrae todo lo que contenga $
precios=[x for x in limpia if '$' in x]
#Extrae todo lo que no contenga $
productos=[x for x in limpia if '$' not in x]
#Dejar solo los registros que comiencen con "precio actual"
precios=[x for x in precios if x.startswith('precio actual')]


In [24]:
#Crear dataframe
walmart=pd.DataFrame({'producto':productos,'precio':precios})
#Incorporar el nombre de la tienda
walmart['tienda']='Walmart'
#Dejar solo números en la columna precio. usar expresiones regulares
walmart['precio']=walmart['precio'].str.extract('(\d+.\d+)')
#Transformar a float
walmart['precio']=walmart['precio'].astype(float)
#Incorporar hora de la consulta
walmart['hora_consulta']=datetime.datetime.now()
walmart

,producto,precio,tienda,hora_consulta
0,Tortilla de maíz por kilo,12.8,Walmart,2023-03-16 15:12:21.974432
1,Tortillas de harina Tía Rosa Tortillinas 561 g,39.5,Walmart,2023-03-16 15:12:21.974432
2,Tortillas de harina Tía Rosa Tortillinas 306 g,22.0,Walmart,2023-03-16 15:12:21.974432
3,Tortillas de harina Tía Rosa Tortillinas liger...,28.0,Walmart,2023-03-16 15:12:21.974432
4,Tortillas Great Value de harina de trigo 572g,24.0,Walmart,2023-03-16 15:12:21.974432
...,...,...,...,...
65,Pan horneado Maretti bruschette chips pizza al...,27.0,Walmart,2023-03-16 15:12:21.974432
66,Relleno para pay Great Value de cereza 595 g,54.0,Walmart,2023-03-16 15:12:21.974432
67,Pan para hot dogs Bimbo Medias noches 8 pzas d...,46.0,Walmart,2023-03-16 15:12:21.974432
68,Pan para hamburguesa 8 pzas,28.0,Walmart,2023-03-16 15:12:21.974432


In [25]:
#Pegar los dataframes
df=pd.concat([chedraui,soriana,walmart],axis=0)
df

,producto,precio,tienda,hora_consulta
0,Tortilla de Maíz Amarillo Kg,12.40,Chedraui,2023-03-16 15:12:01.043505
1,Tortillas Blanca Kg,12.40,Chedraui,2023-03-16 15:12:01.043505
2,Tortilla Harina Tía Rosa 50% Menos Grasa Paque...,18.45,Chedraui,2023-03-16 15:12:01.043505
3,Tortilla Del Barrio Harina Trigo 315g,18.05,Chedraui,2023-03-16 15:12:01.043505
4,Tortillas de Harina Integral Tia Rosa 255g,26.00,Chedraui,2023-03-16 15:12:01.043505
...,...,...,...,...
65,Pan horneado Maretti bruschette chips pizza al...,27.00,Walmart,2023-03-16 15:12:21.974432
66,Relleno para pay Great Value de cereza 595 g,54.00,Walmart,2023-03-16 15:12:21.974432
67,Pan para hot dogs Bimbo Medias noches 8 pzas d...,46.00,Walmart,2023-03-16 15:12:21.974432
68,Pan para hamburguesa 8 pzas,28.00,Walmart,2023-03-16 15:12:21.974432


In [26]:
#Modificar formato de la fecha
df['hora_consulta']=df['hora_consulta'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [27]:
#Guardar en excel
df.to_excel('C:/users/claudio.pacheco/tortilla_webscraping.xlsx',index=False)